In [1]:
import numpy as np
import time
from datetime import timedelta
import os

from PIL import Image

In [2]:
standard_size = 200
num_channels = 3
batch_size_for_transfer_comp = 5

num_color_transf = 2

session = None
image = None
wrap_pre_process_image = None

model = None
transfer_len = 0

In [3]:
def compute_shape_transfer_values(dataset_dir):
    # Lectura de los archivos
    image_filenames = [f for f in os.listdir(dataset_dir) if f.endswith(".jpg")]
    # Ordena todos elementos dentro de la lista
    image_filenames.sort()
    # Nro de elementos del arreglo
    nimages = len(image_filenames)

    # Separar espacio de memoria para las imágenes
    dataset_shape = np.zeros((nimages, standard_size, standard_size,num_channels), dtype=np.float32)
    print (dataset_shape.shape)
    # Separar espacio de memoria para las etiquetas de las imágenes
    cls_shape = np.zeros(nimages, dtype=np.int)

    i = 0 
    current_index = 0
    while i < nimages:
        # batch filenames
        j = min(i+batch_size_for_transfer_comp, nimages)
        batch_filenames = image_filenames[i:j]
        print ("shape transfer values {}".format(current_index))
        
        # Asignar las imágenes correspondientes
        batch_dataset, batch_cls = compute_batch_shape_transfer_values(dataset_dir, batch_filenames)
        print (len(batch_dataset))
        print (len(batch_cls))
        dataset_shape[current_index: current_index + len(batch_cls)] = batch_dataset
        cls_shape[current_index: current_index + len(batch_cls)] = batch_cls
        current_index += len(batch_cls)
        i = j
    return dataset_shape, cls_shape

In [4]:
def compute_batch_shape_transfer_values(dataset_dir, list_filenames):
    # numero de imágenes
    nimages = len(list_filenames)
    # Separar espacio de memoria para las imágenes
    images_shape = np.zeros((nimages, standard_size, standard_size, num_channels), dtype=np.float32)
    # Separar espacio de memoria para las etiquetas de las imágenes
    cls_shape = np.zeros(nimages, dtype=np.int)
    
    current_index = 0
    for image_filename in list_filenames:
        arr_filename = image_filename.split('_') # asumiendo que el formato es class_imagenumber.jpg
        # obtenemos la clase de la imagen
        cls = int(arr_filename[0]) - 1
        # Encontrar la ruta de la imagen
        image_path = dataset_dir + image_filename
        # Manejo la información de la imagen
        pil_image = Image.open(image_path)
        # Resize de la imagen 
        pil_image = pil_image.resize((standard_size, standard_size), Image.ANTIALIAS)
        pad_image = np.asarray(pil_image , dtype=np.float32)
        
        # Se guarda la imagen en el espacio de memoria previamente separado
        images_shape[current_index, :, :, :] = pad_image
        # Se guarda la etiqueta de la clase en el espacio de memoria previamente separado
        cls_shape[current_index] = cls
        current_index += 1
    
    return images_shape, cls_shape

In [5]:
img_dir = ''
output_data = ''
output_cls = ''

In [6]:
img_dir = 'db/test/'
output_data = 'shape_test.npy'
output_cls = 'shape_test_cls.npy'
transfer_values_shape, cls_shape = compute_shape_transfer_values(img_dir)
print (transfer_values_shape.shape)
np.save(output_data, transfer_values_shape)
np.save(output_cls , cls_shape)
print ("Shape saved")

(6, 200, 200, 3)
shape transfer values 0
5
5
shape transfer values 5
1
1
(6, 200, 200, 3)
Shape saved


In [7]:
img_dir = 'db/train/'
output_data = 'shape_train.npy'
output_cls = 'shape_train_cls.npy'
transfer_values_shape, cls_shape = compute_shape_transfer_values(img_dir)
print (transfer_values_shape.shape)
np.save(output_data, transfer_values_shape)
np.save(output_cls , cls_shape)
print ("Shape saved")

(14, 200, 200, 3)
shape transfer values 0
5
5
shape transfer values 5
5
5
shape transfer values 10
4
4
(14, 200, 200, 3)
Shape saved
